In [ ]:
using DrWatson
@quickactivate "IonChannels"

using IonChannelTools
include(srcdir("na9.jl"))
using Plots

In [ ]:
#parameter range

alphas = range(-100,100,1001)

using Plots
plta = plot(title="parameter values from -100 to 100 mV")
for a in alphas
    p = param(a)
    as = ([a a a a a],[a a a a a])
    plta = scatter!(as,p,legend=false,mc="red",ms=2,ma=.1)
    
end

plta

In [ ]:
#steady state range
plts = plot(title="Steady states over voltage -100 to 100 mV")
for a in alphas
    plts = scatter!([a a a a a a a a a],IonChannelTools.steadystate(Gmatrix,a),legend=false,mc="red",ms=2,ma=.1)
end
plts

In [ ]:
#steady states over spike range
dt = 1e-3 #time step in ms
tL = IonChannelTools.spike(200,dt)
plts2 = plot(title="steady states over spike protocol")
for i in 1:size(tL,1)
    if i%500==0
        plts2 = scatter!(fill(tL[i,1],(1,9)),IonChannelTools.steadystate(Gmatrix,tL[i,2]),legend=false,mc="red",ms=2,ma=.1)
    end
end

plts2 = plot!(twinx(),tL[:,1],tL[:,2])

In [ ]:
dt = 1e-3
tL = IonChannelTools.spike(200,dt) #spike
mu0 = IonChannelTools.steadystate(Gmatrix,tL[1,2])
Nadist = IonChannelTools.evolvedist(Gmatrix,tL[:,2],dt)


In [ ]:
pltdis = IonChannelTools.fxnprotplot(Nadist,tL[:,2],tL[:,1])

In [ ]:
Naqex = IonChannelTools.Qex(Nadist,Gmatrix,tL[:,2],dt)


In [ ]:
plt2 = IonChannelTools.fxnprotplot(Naqex,tL[:,2],tL[:,1])

why is it so negative??  
try: recreate thermo graphs from paper with pulse to check...  
figure out which S fro paper to use  

In [ ]:
tL = IonChannelTools.spike(200,dt,"ib") #spike
mu0 = IonChannelTools.steadystate(Gmatrix,tL[1,2])
Nadist = IonChannelTools.evolvedist(Gmatrix,tL[:,2],dt)
Naqex = IonChannelTools.Qex(Nadist,Gmatrix,tL[:,2],dt)
plt3 = IonChannelTools.fxnprotplot(Naqex,tL[:,2],tL[:,1])
plt3 = plot!(title="9-state Q_ex with ib")

In [ ]:
#dQ_ex/dt
function dQex(dists,G,L,dt)
    Qex=[0] #starts at 0
    for i= 1:length(L)-1 #for each step
	newQ = -((transpose(dists[i+1,:]-dists[i,:])*transpose(-log.(IonChannelTools.steadystate(G,L[i]))))[1]) #defined equation term
        Qex = [Qex; newQ] #summation
        end
    return Qex./dt #array of Qex accumulating over the whole interval
    end


In [ ]:
nadQ = dQex(Nadist,Gmatrix,tL[:,2],dt) #dQ/dt, Q rate
#IonChannelTools.Qex(Nadist,Gmatrix,tL[:,2],dt)

In [ ]:
#mask = findall(nadQ.<-10)
#nadQ[mask] .= 0

In [ ]:
function ddt(x,dt)
    dx = Array{Float64}(undef,0,1)
    for i in 1:length(x)-1
        dx = [dx; (x[i+1]-x[i])/dt]
    end
    return dx
end

dqex = ddt(Naqex,dt)

In [ ]:
dqex[findall(abs.(dqex).>.1)] .= 0
#dqex[findall(dqex.>.1)] .= 0

In [ ]:
pltdQ = IonChannelTools.fxnprotplot(dqex,tL[1:end-1,2],tL[1:end-1,1])

In [ ]:
ss = Array{Float64}(undef,0,9)
for i in 1:size(tL,1)
    ss = [ss; IonChannelTools.steadystate(Gmatrix,tL[i,2])]
end #all steady states

In [ ]:
IonChannelTools.fxnprotplot(Nadist-ss,tL[:,2],tL[:,1]) #difference of dist from steady state

In [ ]:
#suprisal
IonChannelTools.fxnprotplot(-log.(ss),tL[:,2],tL[:,1])

**Some steady state entropy**

In [ ]:
dt = 1e-3
tL45 = IonChannelTools.pulse([0 20],dt,[0,0,-45,0])
d1 = IonChannelTools.evolvedist(Gmatrix,tL45[:,2],dt,IonChannelTools.steadystate(Gmatrix,-70)) #-70 mV is "resting potential"

In [ ]:
IonChannelTools.fxnprotplot(IonChannelTools.Qex(d1,Gmatrix,tL45[:,2],dt),tL45[:,2],tL45[:,1])

In [ ]:
#their S_tot entropy calculation

#for a single t
#function S_tot(Gmatrix, V, k, dist)
function S_tot(G,dist)
    #G as actual matrix with values, dist is single row values 
    S_t = 0
#    len = length(Gmatrix(0)[1,:]) #side dimension of matrix
    len = length(G[1,:]) #side dimension of matrix
    for i in 1:len
        for j in 1:len
#            G = Gmatrix(V)
            #kB/2 * sum( (q_ij(V)*p_i - q_ji(V)*p_j)*ln(q_ij/q_ji)
            s1 = (G[i,j]*dist[i]-G[j,i]*dist[j])
            s2 = log(abs((G[i,j]*dist[i])/(G[j,i]*dist[j])))
            if isnan(s2)
                s2 = 0
            end
            s = s1*s2
            S_t += s
        end
    end
    return S_t
end

t = 1 #time index
#S_tot(Gmatrix, tL45[t,1], t, dist)

S = []
for t in 1:length(tL45[:,1])
#    append!(S, S_tot(Gmatrix, tL45[t,1], t, d1))
    append!(S, S_tot(Gmatrix(-45),d1[t,:]))
end
S

In [ ]:
plot(tL45[:,1],S)

In [ ]:
ss = Array{Float64}(undef,0,9)
for i in 1:size(tL45,1)
    ss = [ss; IonChannelTools.steadystate(Gmatrix,tL45[i,2])]
end #all steady states
IonChannelTools.fxnprotplot(d1-ss,tL45[:,2],tL[:,1]) #difference of dist from steady state

In [ ]:
dt = 1e-3
tL30 = IonChannelTools.pulse([0 20],dt,[0,0,-30,0])
d2 = IonChannelTools.evolvedist(Gmatrix,tL30[:,2],dt,IonChannelTools.steadystate(Gmatrix,-70)) #-70 mV is "resting potential"
S2 = []
for t in 1:length(tL30[:,1])
    append!(S2, S_tot(Gmatrix(-30),d2[t,:]))
end
ss = Array{Float64}(undef,0,9)
for i in 1:size(tL30,1)
    ss = [ss; IonChannelTools.steadystate(Gmatrix,tL30[i,2])]
end #all steady states
IonChannelTools.fxnprotplot(d2-ss,tL30[:,2],tL30[:,1]) #difference of dist from steady state

In [ ]:
dt = 1e-3
tL15 = IonChannelTools.pulse([0 20],dt,[0,0,15,0])
d3 = IonChannelTools.evolvedist(Gmatrix,tL15[:,2],dt,IonChannelTools.steadystate(Gmatrix,-70)) #-70 mV is "resting potential"
S3 = []
for t in 1:length(tL15[:,1])
    append!(S3, S_tot(Gmatrix(15),d3[t,:]))
end
ss = Array{Float64}(undef,0,9)
for i in 1:size(tL15,1)
    ss = [ss; IonChannelTools.steadystate(Gmatrix,tL15[i,2])]
end #all steady states
IonChannelTools.fxnprotplot(d3-ss,tL15[:,2],tL15[:,1]) #difference of dist from steady state